# BBoxMaskPose Quickstart

This notebook demonstrates the public API for both PMPose and BBoxMaskPose.

## Installation Reminder

Before running this notebook, ensure you have installed BBoxMaskPose:

```bash
# Install dependencies
pip install -U openmim
mim install mmengine "mmcv==2.1.0" "mmdet==3.3.0" "mmpretrain==1.2.0"
pip install -r requirements.txt

# Install in editable mode
pip install -e .

# Download SAM2 weights
bash models/SAM/download_ckpts.sh
```

## Part 1: PMPose - Pose Estimation with Bounding Boxes

PMPose (currently MaskPose) performs pose estimation given an image and bounding boxes.

In [ ]:
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Import PMPose public API
from pmpose import PMPose

### Initialize PMPose Model

In [ ]:
# Initialize PMPose with pretrained weights
pose_model = PMPose(
    device="cuda",           # Use 'cpu' if no GPU available
    variant="default",       # Default MaskPose-b model
    from_pretrained=True,    # Download pretrained weights
)

print("✓ PMPose model initialized")

### Load Image and Define Bounding Boxes

In [ ]:
# Load example image
image_path = "demo/data/004806.jpg"
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display image
plt.figure(figsize=(12, 8))
plt.imshow(img_rgb)
plt.title("Input Image")
plt.axis('off')
plt.show()

# Define bounding boxes for people in the image [x1, y1, x2, y2]
bboxes = np.array([
    [180, 100, 380, 500],   # Person 1
    [350, 150, 550, 500],   # Person 2
    [500, 120, 700, 480],   # Person 3
], dtype=np.float32)

print(f"Image shape: {img.shape}")
print(f"Number of bboxes: {len(bboxes)}")

### Run Pose Estimation

In [ ]:
# Run pose estimation
keypoints, presence, visibility, heatmaps = pose_model.predict(
    image=img,
    bboxes=bboxes,
    masks=None,              # Optional instance masks
    return_probmaps=False,   # Set True to get heatmaps
)

print(f"Keypoints shape: {keypoints.shape}")       # (N, K, 3) - [x, y, score]
print(f"Presence shape: {presence.shape}")         # (N, K) - presence probability
print(f"Visibility shape: {visibility.shape}")     # (N, K) - visibility flag

# Print average confidence per person
for i in range(len(keypoints)):
    avg_conf = keypoints[i, :, 2].mean()
    print(f"Person {i+1}: average confidence = {avg_conf:.3f}")

### Visualize Results

In [ ]:
# Visualize pose estimation results
vis_img = pose_model.visualize(
    image=img,
    keypoints=keypoints,
    bboxes=bboxes,
)

# Display
vis_img_rgb = cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12, 8))
plt.imshow(vis_img_rgb)
plt.title("PMPose Results")
plt.axis('off')
plt.show()

## Part 2: BBoxMaskPose - Full Detection + Pose + Segmentation Pipeline

BBoxMaskPose runs the complete pipeline: detection → pose estimation → SAM refinement.

In [ ]:
# Import BBoxMaskPose public API
from bboxmaskpose import BBoxMaskPose

### Option 1: BMP with Internal Pose Model

In [ ]:
# Initialize BBoxMaskPose (creates internal PMPose)
bmp_model = BBoxMaskPose(
    config="BMP_D3",         # BMP configuration
    device="cuda",           # Use 'cpu' if no GPU
    pose_model=None,         # Let BMP create internal model
)

print("✓ BBoxMaskPose initialized with internal pose model")

### Run Full Pipeline

In [ ]:
# Run full BMP pipeline
result = bmp_model.predict(
    image=image_path,
    bboxes=None,             # Let detector find bboxes
    return_intermediates=False,
)

print(f"Detected {len(result['bboxes'])} people")
print(f"Keypoints shape: {result['keypoints'].shape}")
print(f"Masks shape: {result['masks'].shape}")
print(f"Presence shape: {result['presence'].shape}")
print(f"Visibility shape: {result['visibility'].shape}")

### Visualize BMP Results

In [ ]:
# Visualize complete results
vis_img = bmp_model.visualize(
    image=image_path,
    result=result,
)

# Display
vis_img_rgb = cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12, 8))
plt.imshow(vis_img_rgb)
plt.title("BBoxMaskPose Results")
plt.axis('off')
plt.show()

### Option 2: BMP with External PMPose Model

You can also create a PMPose model separately and inject it into BMP.

In [ ]:
# Create PMPose model first
my_pose_model = PMPose(
    device="cuda",
    variant="default",
    from_pretrained=True,
)

# Inject into BBoxMaskPose
bmp_model_with_external_pose = BBoxMaskPose(
    config="BMP_D3",
    device="cuda",
    pose_model=my_pose_model,   # Use our PMPose instance
)

print("✓ BBoxMaskPose initialized with external pose model")

# Run pipeline (same as before)
result2 = bmp_model_with_external_pose.predict(
    image=image_path,
    bboxes=None,
)

print(f"Detected {len(result2['bboxes'])} people")

## Summary

This notebook demonstrated:

1. **PMPose API**: Pose estimation with bounding boxes
   - `PMPose()` initialization
   - `predict()` for inference
   - `visualize()` for visualization

2. **BBoxMaskPose API**: Full detection + pose + segmentation pipeline
   - Internal pose model creation
   - External pose model injection
   - `predict()` for full pipeline
   - `visualize()` for results

Both APIs provide stable, easy-to-use interfaces while maintaining backward compatibility with the underlying MaskPose model.